# Programación Dinámica

Vamos a ver una aplicación más compleja y con muchos más requerimientos. 

Intentaremos resolver el problema de "subset selection" en una regresión lineal múltiple.

Recordemos un poco de teoría.

Buscamos unos pesos $\beta_0,...,\beta_n$ que describan la relación entre una variable dependiente $y$ y las variables independientes $x_1,...,x_n$:

\begin{equation}
	y=\beta_0+\beta_1 x_1 + \beta_2 x_2 + ... + \beta_n x_n + \mathcal{N}(0,\sigma)
\end{equation}

Encontrar el mejor estimador consiste en resolver un problema de mínimos cuadrados:

\begin{equation}
\beta=(X^T X)^{-1} X^T y
\end{equation}

El problema es que algunas de la contribución de algunas variables independientes puede que no sea estadísticamente significativa o que no contribuya al objetivo final de una regresión (que consiste en explicar un fenómeno).

Existen diversos métodos para escoger el mejor conjunto de variables $R^2_{adj}$, $C_P$ de Mallows, AIC, BIC, ...

Utilizaremos PD para generar múltiples alternativas para cada posible cardinalidad del conjunto de predictores. 

Nota: La implementación es "quick-and-dirty", hay varias cosas que deben mejorarse (p.ej. reordenar las variables).

In [ ]:
#  el número de variables explicativas **nPredictores**
#  un vector **y** de variable a explicar
#  una matriz **x** de variables explicativas, se construye columna a columna usando variables aleatorias
#  calcula la suma de cuadrados totales usando aritmética vectorial
#  esta función debería sustituirse por la lectura de datos desde un fichero
function init_data()
    nObservaciones=25
    nPredictores=15
    x=ones(nObservaciones) #variable independiente
    for i in 1:nPredictores
        x = [x rand(nObservaciones) ]
    end
    y = rand(nObservaciones)
    calculoMedia= Array(Float64,nObservaciones)
    fill!(calculoMedia,mean(y))
    calculoMedia = calculoMedia .- y #variación respecto de la media
    # al hacer dot(calculoMedia,calculoMedia) me reporta la suma de cuadrados de las desviaciones
    nObservaciones,nPredictores,x, y,dot(calculoMedia,calculoMedia)
end

El cálculo del estimador OLS corresponde a resolver un sistema de ecuaciones (ver https://docs.julialang.org/en/latest/stdlib/linalg/#Base.:\\-Tuple{AbstractArray{T,2}%20where%20T,Union{AbstractArray{T,1},%20AbstractArray{T,2}}%20where%20T} para algunos detalles)

In [ ]:
function determinarR2(nObservaciones,nPredictores,X,y,SStot)
    #println("\nX: ",X)
    #println("\ny: ",y)
    b = X\y #Aquí calculamos estimadores
    #println("\nb:",b)
    estimados= X * b
    error= dot(y-estimados,y-estimados)
    1.0-error/SStot
end

In [ ]:
# función que selecciona un subconjunto de la matriz X.
# input:
#    matriz: la matriz de variables explicativas
#    nPredictores: el número total de variables explicativas que podemos seleccionar (se usa para recorrer por todas y ver si son activas)
#    seleccion: vector 0/1 que tiene valor igual a 1 si la variable debe estra dentro del modelo
#
# output:
#    ret: matriz con el subconjunto de variables a usar
#
# notas:
#   al menos uno de los coeficientes de selección debe valer 1 para crear una matriz de dos dimensiones, en caso contrario el codigo falla
#   Siempre se considera que hay término independiente en el cálculo, que corresponde a la posición 1 de la matriz (columna de 1's)
#   no es lo más "limpio" pero como proof-of-concept es suficiente
function sub_data(matriz,nPredictores,seleccion)
    count=1
    ret = matriz[:,1]
    for i in 1:nPredictores
        if seleccion[i]==1
            ret=vcat(ret,matriz[:,i+1])
            count += 1
        end
    end
    ret=reshape(ret,size(matriz,1),count)
    ret
end

## Estructura del programa dinámico

La estructura de la recurrencia es la siguiente.

Cada etapa está asociado al número de variables seleccionadas como predictores (etapa 1, una variable seleccionada, etapa 2, dos, y así sucesivamente). La transición consiste en unir una variable al conjunto de predictores. Evidentemente el número de estados por etapa es combinatorio y rápidamente sale de dimensiones (por ejemplo, para un problema con el número indicado de predictores, el número de estados por etapa es:

In [ ]:
nPredictores=20 #21 da error por overflow!
for i in 1:nPredictores
    println("i: ",i,"\t",factorial(nPredictores)/(factorial(i)*factorial(nPredictores-i)))
    end

La cantidad de estados (y de mínimos cuadrados que hay que resolver, que es todavía mayor) indica que vamos a tener que optar por limitar el número de estados que conservamos en cada etapa (lógicamente nos quedaremos con aquellos estados que ofrezcan mejor R^2).

Esa es la función que realiza maxNumEnPQ. Sólo usamos los maxNumEnPQ mejores de la etapa $n$ para generar estados de la etapa $n+1$.

La estructura del algoritmo se asemeja a (Uncapacitated Lot Sizing. Método 2. Estados asociados al nivel de inventario y recurrencia forward) que ya vimos. Tenemos una lista que tiene estados de la etapa $n$ que (pqAnterior) que se utiliza para generar estados de la etapa $n+1$ (pqPosterior).

Se desarrollan todas transiciones de cada estado de pqAnterior y se almacenan en: 

* pqPosterior (si son mejores que las maxNumEnPQ mantenidas en la estructura) 
* enPQ (que contiene todos los estados que ya han sido evaluados para evitar repetir evaluación)

Cuando no quedan estados en pqAnterior, quiere decir que puedo pasar a desarrollar la etapa siguiente (esto es, pqPosterior se transforman en pqAnterior).

El uso de una cola de prioridad (pq) para contener los estados de cada etapa es especialmente importante. La cola de prioridad me permitirá detectar el peor estado (en términos de $R^2$) rápidamente, eliminarlo de la lista y guardar nuevos estados según su calidad (ordenados en este caso de menor a mayor PQ para mejorar eficiencia eliminando y consultando el valor del peor).

In [ ]:
function main()   #inicio del programa principal
    srand(0) #inicializa la semilla aleatoria para reproducibilidad
    maxNumEnPQ = 500 #valor de control de uso de memoria en el algoritmo
    nObservaciones,nPredictores,X, y,SStot = init_data()
    println("SStot: ",SStot)
    selector = zeros(Int,nPredictores)

    enPQ = Dict()
    numEnPQ = 0
    pqAnterior = Collections.PriorityQueue(Array{Int},Float64)
    Collections.enqueue!(pqAnterior,selector,0.0)
    pqPosterior = Collections.PriorityQueue(Array{Int},Float64)
    etapa = 1
    for etapa in 1:nPredictores
        @printf("etapa: %d  ",etapa)
        println(Collections.length(pqAnterior))
        #se van seleccionando los vertices en pAnterior, se desarrollan y se dejan en pqPosterior
        while Collections.isempty(pqAnterior) == false       
            selector = Collections.dequeue!(pqAnterior)
            for i in 1:nPredictores
                if selector[i] == 0 #si no se había escogido
                    selector[i] = 1 # lo incluyo en la lista
                    retorno=get!(enPQ,selector,0)
                    if(retorno==0)  #no está en la memoria (es nueva) y hay que mirar si se guarda
                        #la guardo en el diccionario de referencia rápida con un 1 que significa que está
                        enPQ[copy(selector)]=1 
                        Xalterado = sub_data(X,nPredictores,selector) #determino r2 (paso 1)
                        r2 = determinarR2(nObservaciones,nPredictores,Xalterado,y,SStot) #determino r2 (paso 2)
                        #println("r2: ",r2)
                        if numEnPQ < maxNumEnPQ  #lo guardo siempre porque faltan alternativas
                            Collections.enqueue!(pqPosterior,deepcopy(selector),r2)
                            numEnPQ += 1
                        else #tengo que ver si es mejor que el peor en PQ
                            descarte, comparacion = Collections.peek(pqPosterior) #leo el valor de la peor
                            if comparacion < r2 #si es mejor, entonces elimino la peor y añado la nueva
                                Collections.dequeue!(pqPosterior)
                                Collections.enqueue!(pqPosterior,deepcopy(selector),r2)
                            end
                        end
                    end
                    selector[i]=0 #para volver a inicio
                end
            end #for
        end #while
      
        while isempty(pqPosterior) == false
            selector,valor = Collections.peek(pqPosterior)
            println("selector: ",selector," valor: ",valor)
            Collections.enqueue!(pqAnterior,Collections.dequeue!(pqPosterior),0.0)
        end
        Collections.empty!(enPQ)       # y hay que vaciar la memoria
        numEnPQ = 0
    end
end

In [ ]:
main()